<a href="https://colab.research.google.com/github/urszkam/AoC_2022/blob/main/Copy_of_rozwi%C4%85zania_dask_zadania.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://students.mimuw.edu.pl/~mw404851/dane.zip

In [ ]:
!mkdir dane && unzip dane.zip -d dane

In [ ]:
import dask.dataframe as dd

df = dd.read_parquet(f"dane/data_part_*.gz.parquet", engine='pyarrow')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.info()

Na podstawie treści prezentacji dotyczącej Dask, oto kilka zadań praktycznych, które mogą pomóc w zrozumieniu jego funkcji i możliwości:

---

### **Zadanie 1: Wczytywanie danych i partycjonowanie**
1. Wczytaj zbiór danych z pliku **Parquet** (lub CSV, jeśli nie masz Parquet) do Dask DataFrame.
2. Sprawdź liczbę partycji w swoim DataFrame.
3. Zoptymalizuj liczbę partycji, dopasowując je do zasobów systemowych (np. ustaw 5 partycji) używajac repartiton.


In [ ]:
df

In [ ]:
df.tail()

In [ ]:
df = df.repartition(npartitions=5)
df



### **Zadanie 2: Operacje na partycjach**
1. Użyj metody `map_partitions`, aby obliczyć liczbę wierszy w każdej partycji.
2. Zsumuj wszystkie wartości w kolumnie `tip` z każdej partycji za pomocą `reduce`.

**Cel:** Nauka operacji map-reduce w Dask.

---

In [ ]:
rows_per_partition = df.map_partitions(len).compute()
rows_per_partition

In [ ]:
result = df['total_bill'].sum().compute()
result

In [ ]:
rows_per_partition = df.map_partitions(len).compute()
rows_per_partition



### **Zadanie 3: Filtrowanie i grupowanie**
1. Wczytaj dane dotyczące napiwków (`tips`).
2. Przefiltruj dane, pozostawiając tylko wiersze, w których `time` to `'Dinner'`.
3. Użyj `groupby`, aby obliczyć średnią wartości `total_bill` dla każdej wartości `day`.

**Cel:** Poznanie podstawowych operacji na danych w Dask.

---

In [ ]:
df[df['time']=='Dinner'].compute()

In [ ]:
df[df['tip']]

In [ ]:
df.groupby('day').total_bill.mean().compute()



### **Zadanie 4: Lazy Evaluation**
1. Utwórz operację obliczającą sumę kolumny `total_bill`, ale nie wykonuj jej natychmiast (lazy evaluation).
2. Zwizualizuj graf obliczeń za pomocą metody `visualize`.


---

In [ ]:
result = df['total_bill'].sum().visualize()
result

### **Zadanie 5: Wizualizacja danych**
Oblicz średnią wartość rachunku (total_bill) dla każdej kategorii rozmiaru grupy (size) i przedstaw wynik na wykresie słupkowym.

In [ ]:
avg_total_bill_by_size = df.groupby('size')['total_bill'].mean().compute()
avg_total_bill_by_size

In [ ]:
import matplotlib.pyplot as plt
avg_total_bill_by_size = df.groupby('size')['total_bill'].mean().compute()

avg_total_bill_by_size.plot(kind='bar')
plt.xlabel('Rozmiar grupy (size)')
plt.ylabel('Średnia wartość rachunku (total_bill)')
plt.title('Średnia wartość rachunku w zależności od rozmiaru grupy')
plt.xticks(rotation=0)
plt.show()

### Zadanie 6: Tworzenie nowych kolumn
1. Dodaj nową kolumnę tip_percentage, która oblicza procent napiwku w stosunku do rachunku (tip / total_bill * 100).
2. Policz średni procent napiwku dla każdej kombinacji płci (sex) i dnia tygodnia (day).

In [ ]:
df['tip_percentage'] = (df['tip'] / df['total_bill']) * 100
df['tip_percentage'].compute()

In [ ]:
# Policz średni procent napiwku
# dla każdej kombinacji płci (sex) i dnia tygodnia (day).

avg_tip_sex = df.groupby('sex')['tip_percentage'].mean().compute()
avg_tip_sex

In [ ]:
avg_tip_day = df.groupby('day')['tip_percentage'].mean().compute()
avg_tip_day

### average_bill_by_day_dask = {}  Zadanie: Praca z Dummy Variables w Dask
1. Utwórz zmienne dummy dla kolumny day (dzień tygodnia) w zbiorze tips.
2. Połącz je z oryginalnym zestawem danych.
3. Oblicz średni rachunek (total_bill) dla każdego dnia tygodnia, korzystając z wygenerowanych zmiennych dummy.

In [ ]:
df['day'].unique().compute()

In [ ]:
df = df.categorize().compute()
df.head()

In [ ]:
# Tworzenie zmiennych dummy
day_dummies = dd.get_dummies(df['day'], prefix='day')

df_with_dummies = dd.concat([df, day_dummies], axis=1)
df_with_dummies.head()

# Średni rachunek na podstawie dummy variables
mean_total_bill_by_day = {}
for day in day_dummies.columns:
    mean = ((df_with_dummies['total_bill'] * df_with_dummies[day]).sum() / df_with_dummies[day].sum()).compute()
    mean_total_bill_by_day[day] = mean

print("Średni total_bill dla każdego dnia (dummy):")
for day, mean in mean_total_bill_by_day.items():
    print(f"{day}: {mean}")




In [ ]:
df_with_dummies.head()

In [ ]:
# Verify against the previous groupby result
print("\nVerification using direct groupby:")
print(df.groupby('day').total_bill.mean())